In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from decision_tree.decision_tree import MyDecisionTree

## Загрузка и анализ данных

In [2]:
data = load_wine()
X, y = data.data, data.target[:, None]
print(X.shape)
print(y.shape)

(178, 13)
(178, 1)


In [3]:
df = pd.DataFrame(
    data = np.concatenate((data.data, data.target[:, None]), axis=1), 
    columns = [*data.feature_names, 'target'])
df['target'] = df['target'].astype(int)
df.sample(5)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
154,12.58,1.29,2.10,20.0,103.0,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640.0,2
46,14.38,3.59,2.28,16.0,102.0,3.25,3.17,0.27,2.19,4.90,1.04,3.44,1065.0,0
67,12.37,1.17,1.92,19.6,78.0,2.11,2.00,0.27,1.04,4.68,1.12,3.48,510.0,1
36,13.28,1.64,2.84,15.5,110.0,2.60,2.68,0.34,1.36,4.60,1.09,2.78,880.0,0
142,13.52,3.17,2.72,23.5,97.0,1.55,0.52,0.50,0.55,4.35,0.89,2.06,520.0,2


Проверим данные на корректность:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  targe

In [5]:
df.target.value_counts() # Подсчет классов в датасете

1    71
0    59
2    48
Name: target, dtype: int64

## Тестирование (нудное и скучное, прямо как моя жизнь)

In [6]:
RANDOM_STATE = 42
X = np.ones((4, 5), dtype=float) * np.arange(4)[:, None]
y = np.arange(4)[:, None] + np.asarray([0.2, -0.3, 0.1, 0.4])[:, None]
class_estimator = MyDecisionTree(max_depth=10, criterion_name='gini')

(X_l, y_l), (X_r, y_r) = class_estimator.make_split(1, 1., X, y)

assert np.array_equal(X[:1], X_l)
assert np.array_equal(X[1:], X_r)
assert np.array_equal(y[:1], y_l)
assert np.array_equal(y[1:], y_r)

In [7]:
digits_data = load_digits().data
digits_target = load_digits().target[:, None] # to make the targets consistent with our model interfaces
X_train, X_test, y_train, y_test = train_test_split(digits_data, digits_target, test_size=0.2, random_state=RANDOM_STATE)

In [8]:
assert len(y_train.shape) == 2 and y_train.shape[0] == len(X_train)

In [10]:
class_estimator = MyDecisionTree(max_depth=10, criterion_name='gini')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
accuracy_gini = accuracy_score(y_test, ans)
print(accuracy_gini)

AttributeError: 'MyDecisionTree' object has no attribute 'height'

In [38]:
reference = np.array([0.09027778, 0.09236111, 0.08333333, 0.09583333, 0.11944444,
       0.13888889, 0.09930556, 0.09444444, 0.08055556, 0.10555556])

In [39]:
class_estimator = MyDecisionTree(max_depth=10, criterion_name='entropy')
class_estimator.fit(X_train, y_train)
ans = class_estimator.predict(X_test)
accuracy_entropy = accuracy_score(y_test, ans)
print(accuracy_entropy)

/home/shredder/University/DSS_Part_2/decision_tree/decision_tree.py:8: RuntimeWarning: invalid value encountered in true_divide
  p = np.sum(y, axis=0) / n # probability of each class


KeyboardInterrupt: 

In [ ]:
assert  0.84 < accuracy_gini < 0.9
assert  0.86 < accuracy_entropy < 0.9
assert np.sum(np.abs(class_estimator.predict_proba(X_test).mean(axis=0) - reference)) < 1e-4

AssertionError: 

## Обучение и предсказание

In [12]:
model = MyDecisionTree(n_classes=3, max_depth=5)
model.fit(X, y)

/home/shredder/University/DSS_Part_2/decision_tree/decision_tree.py:14: RuntimeWarning: invalid value encountered in true_divide
  p = np.sum(y, axis=0) / n # probability of each class


In [16]:
preds = model.predict(X)
print(preds)

AttributeError: 'NoneType' object has no attribute 'predicted_value'